In [1]:
import numpy as np
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss, MSELoss
from pennylane import NesterovMomentumOptimizer
from thesis.qcnn import QCNN

dims = (16, 16), (28, 28), (32, 32)
num_trials = 3

In [2]:
from thesis.quantum.operation.ansatz.convolution.v3 import ConvolutionAnsatz as A
from thesis.ml.data import BinaryData
from thesis.ml.optimize import Optimizer
from thesis.ml.data import image_transform
from thesis.experiment import Experiment
from thesis.cnn import CNN

dims = (16, 16)

data = BinaryData(FashionMNIST, image_transform(dims, flatten=True), batch_size=(80, 1000))
# optimizer = Optimizer(SGD, lr=0.01, momentum=0.9, nesterov=True)
optimizer = Optimizer(Adam)
qcnn = QCNN.with_logging(data, optimizer, CrossEntropyLoss(), epoch=1)

# qcnn.ansatz = A.from_dims(dims, num_layers=2)
# qcnn.draw()
    
experiment = Experiment(qcnn, num_trials, results_schema=["accuracy"])
# experiment(dims, 1)
experiment(A, dims, num_layers=2)

experiment.draw()

2023-06-02 04:19:38,044: (qcnn_trial_0) Number of Parameters: 66
